In [1]:
import tensorflow as tf
import qriemannopt as qro
import matplotlib.pyplot as plt

C:\Users\HYPERPC\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\HYPERPC\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
U = tf.complex(tf.random.normal((4, 4)),
              tf.random.normal((4, 4)))
U, _ = tf.linalg.qr(U)
Z = tf.complex(tf.random.normal((8, 8)),
              tf.random.normal((8, 8)))
Z, _ = tf.linalg.qr(Z)
Z = Z[:, :4]

# pauli matrices
sigma = tf.constant([[[1j*0, 1 + 1j*0], [1 + 1j*0, 0*1j]],
                    [[0*1j, -1j], [1j, 0*1j]],
                    [[1 + 0*1j, 0*1j], [0*1j, -1 + 0*1j]]],dtype=tf.complex128)

# hamiltonian
h = -tf.einsum('ij,kl->ikjl', sigma[2], sigma[2]) +\
tf.einsum('ij,kl->ikjl', -h_x * sigma[0] / 2, tf.eye(2, dtype=tf.complex128)) +\
tf.einsum('ij,kl->ikjl', -tf.eye(2, dtype=tf.complex128), h_x * sigma[0] / 2)

In [ ]:
@tf.function
def renorm(U, Z, h):
    """the function renormalize local operation. As an input
    it takes disentangler, isometry and local operator.
    --------------------------------------------------------
    U.shape = (chi ** 2, chi ** 2)
    Z.shape = (chi ** 3, chi_new)
    returns tensor of shape (chi_new, chi_new, chi_new, chi_new)"""
    # U and Z reshaping
    chi = tf.cast(tf.math.sqrt(tf.cast(U.shape[0], dtype=tf.float64)), dtype=tf.int32)
    chi_new = Z.shape[-1]
    U_resh = tf.reshape(U, (chi, chi, chi, chi))
    Z_resh = tf.reshape(Z, (chi, chi, chi, chi_new))
    # applying disentangler
    uhu_up = tf.einsum('jmqr,ijkl,lmnp->iqrknp', tf.math.conj(U_resh), h, U_resh) ##
    uhu_mid = tf.einsum('ijnm,ijkl,klqp->nmqp', tf.math.conj(U_resh), h, U_resh) ##
    uhu_down = tf.einsum('minr,ijlk,mlqp->nrjqpk', tf.math.conj(U_resh), h, U_resh) ##
    # applying isometries
    z_up = tf.einsum('ijmn,ijkl->nmlk', tf.math.conj(Z_resh), Z_resh) ##
    z_down = tf.einsum('ijmn,kjml->nilk', tf.math.conj(Z_resh), Z_resh) ##
    aux_up = tf.einsum('qijp,ijklmn,qlmo->pkon', tf.math.conj(Z_resh), uhu_up, Z_resh) ##
    h_up = tf.einsum('ijkl,pjql->ipkq', aux_up, z_down) ##
    aux_down = tf.einsum('jkqp,ijklmn,mnqr->iplr', tf.math.conj(Z_resh), uhu_down, Z_resh) ##
    h_down = tf.einsum('ijkl,jqlp->iqkp', z_up, aux_down) ##
    # total renormalized hamiltonian
    h_mid = tf.einsum('ijkl,jqlm,rqom->irko', z_up, uhu_mid, z_down)
    return (1 / 3) * (h_up + h_mid + h_down)